# Example Notebook

## Example 1: Building YAML config from a Python dictionary
First, we need to import `uwtools.api.config` from the uwtools python package.

In [45]:
from uwtools.api import config

The `config.get_yaml_config` method can create a `YAMLconfig` object when given a Python dictionary.

In [46]:
# An example Python dictionary
dictionary = {"values":{"date":20240105, "greeting":"Good Night", "recipient":"Moon", "repeat":2}}

# Build a YAMLconfig object from the dictionary
config_yaml = config.get_yaml_config(dictionary)
print(config_yaml)

values:
  date: 20240105
  greeting: Good Night
  recipient: Moon
  repeat: 2


## Example 2: Rendering a template with uwtools
Next, let's look at using the `template` tool to render a Jinja2 template.

In [47]:
from uwtools.api import template

We have a Jinja2 template file in `fixtures/user_config.yaml` that looks like this:

In [48]:
%%bash
cat ./fixtures/user_config.yaml

user:
  name: {{ first }} {{ last }}
  favorite_food: {{ food }}


We can use another yaml file that contains the values we want to add to the template to complete it:

In [49]:
%%bash
cat ./fixtures/user_values.yaml

first: John
last: Doe
food: burritos


Using `template.render` we can render the `user_config.yaml` file using the values supplied by the `user_values.yaml` to create a complete and ready to use config file.

In [50]:
# path of Jinja2 template to update
source = "./fixtures/user_config.yaml"

# values to add
vals = "./fixtures/user_values.yaml"

# destination of the rendered file
target = "./fixtures/rendered_config.yaml"

# render the template 
print(template.render(values_src=vals, values_format="yaml", input_file=source, output_file=target))

user:
  name: John Doe
  favorite_food: burritos


Let's take a look at the rendered file:

In [51]:
%%bash
cat ./fixtures/rendered_config.yaml

user:
  name: John Doe
  favorite_food: burritos


## Example 3: Comparing two config files
Let's explore using the `config.compare()` method to compare two two config files.

We again need to start by importing the `uwtools.api.config` from the uwtools python package.

In [52]:
from uwtools.api import config

The `config.compare()` method has the following syntax:

`uwtools.api.config.compare(config_1_path: Path | str, config_2_path: Path | str, config_1_format: str | None = None, config_2_format: str | None = None)→ bool`

Recognized file extensions are: ini, nml, sh, yaml.

Parameters:

- config_1_path – Path to 1st config file

- config_2_path – Path to 2nd config file

- config_1_format – Format of 1st config file (optional if file’s extension is recognized)

- config_2_format – Format of 2nd config file (optional if file’s extension is recognized)

The method returns False if config files had differences, otherwise True.

For example, let's compare two Fortran namelist files with differences:

In [42]:
# path of nml file that contains 'greeting = "Hello" recipient = "World"'
file_a = "./fixtures/config_test_file_a.nml"

# path of nml file that contains 'greeting = "Hello" recipient = "Different World"'
different_file_b = "./fixtures/config_test_file_b.nml"

config.compare(file_a, different_file_b)

False

The `config()` method returns `False` to denote the files are different.

Now to compare two files that contain identical text:

In [43]:
# path of nml file that contains 'greeting = "Hello" recipient = "World"'
file_a = "./fixtures/config_test_file_a.nml"

# path of nml file that contains 'greeting = "Hello" recipient = "World"'
identical_file_c = "./fixtures/config_test_file_c.nml"

config.compare(file_a, identical_file_c)

True

The `config()` method returns `True` to denote the files are semantically equivalent.